In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

Age, Cabin, Embarked 에 NULL값이 있다 
다른 Column 값들은 Non-NULL 개수가 891

## 1. Sex

In [ ]:
print (train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean())

여자는 74% 생존
남자는 18% 생존

## 2. Pclass

In [ ]:
print (train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

1등급 객실 62% 생존 2등급 47% 3등급 24% 생존

In [ ]:
pd.crosstab([train.Sex,train.Survived],train.Pclass,margins=True).style.background_gradient(cmap='summer_r')

1등급 객실의 여성생존자가 가장 많고 3등급객실의 남성사망자가 제일 많다

## 3. Age

In [ ]:
g = sns.FacetGrid(train, col='Survived')
g = g.map(sns.distplot, "Age")

## 4.Pclass,Sex,Age

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
sns.violinplot('Pclass','Age',hue='Survived',data=train,scale='count',split=True,ax=ax[0])
ax[0].set_title('Pclass & Age')
ax[0].set_yticks(range(0,110,10))

sns.violinplot('Sex','Age',hue='Survived',data=train,scale='count',split=True,ax=ax[1])
ax[1].set_title('Sex & Age')
ax[1].set_yticks(range(0,110,10))

plt.show()


## 5. Embarked

In [ ]:
f,ax=plt.subplots(1,1,figsize=(7,7))
train[['Embarked','Survived']].groupby(['Embarked'],as_index=True).mean().sort_values(by='Survived',ascending=False).plot.bar(ax=ax)

In [ ]:
train[['Embarked','Survived']].groupby(['Embarked'],as_index=True).mean().sort_values(by='Survived')

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,15))
sns.countplot('Embarked',data=train,ax=ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Board')


sns.countplot('Embarked', hue='Sex',data=train,ax=ax[0,1])
ax[0,1].set_title('(2) Male-Female split for Embarked')

sns.countplot('Embarked', hue='Survived',data=train,ax=ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')

sns.countplot('Embarked', hue='Pclass', data=train,ax=ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')

#상하좌우간격
plt.subplots_adjust(wspace=0.2, hspace=0.5)

plt.show()

## 6.Feature Engineering

In [ ]:
train['Age'].isnull().sum()

In [ ]:
train['Embarked'].isnull().sum()

In [ ]:
train['Embarked'].fillna('S',inplace=True)

In [ ]:
train['Embarked'].isnull().sum()

In [ ]:
train['Age'].mean()

str.extract() 원하는 문자열 추출  Mr. Mrs. 등을 찾기 위해서!

In [ ]:
train['first']=train['Name'].str.extract('([A-Za-z]+)\.')
test['first']=test['Name'].str.extract('([A-Za-z]+)\.')

In [ ]:
train['Name'].str.extract('([A-Za-z]+)\.')

In [ ]:
pd.crosstab(train['first'],train['Sex']).T.style.background_gradient(cmap='summer_r')

In [ ]:
train['first'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
                      ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)

In [ ]:
train.groupby('first')['Survived'].mean().plot.bar()

In [ ]:
train.groupby('first').mean()

In [ ]:
train.loc[(train['Age'].isnull())&(train['first']=='Master'),'Age']=7
train.loc[(train['Age'].isnull())&(train['first']=='Miss'),'Age']=23
train.loc[(train['Age'].isnull())&(train['first']=='Mr'),'Age']=32
train.loc[(train['Age'].isnull())&(train['first']=='Mrs'),'Age']=35
train.loc[(train['Age'].isnull())&(train['first']=='Other'),'Age']=46

test.loc[(test['Age'].isnull())&(test['first']=='Master'),'Age']=7
test.loc[(test['Age'].isnull())&(test['first']=='Miss'),'Age']=23
test.loc[(test['Age'].isnull())&(test['first']=='Mr'),'Age']=32
test.loc[(test['Age'].isnull())&(test['first']=='Mrs'),'Age']=35
test.loc[(test['Age'].isnull())&(test['first']=='Other'),'Age']=46

In [ ]:
train['first'].unique()

In [ ]:
def category_age(x):
    if x<10:
        return 0
    elif x<20:
        return 1
    elif x<30:
        return 2
    elif x<40:
        return 3
    elif x<50:
        return 4
    elif x<60:
        return 5
    elif x<70:
        return 6
    else:
        return 7

In [ ]:
train['Age_1']=train['Age'].apply(category_age)

In [ ]:
train.head()

In [ ]:
train.drop(['Age'],axis=1,inplace=True)
test.drop(['Age'],axis=1,inplace=True)

In [ ]:
train['first']=train['first'].map({'Master':0,'Miss':1,'Mrs':3,'Other':4})
test['first']=test['first'].map({'Master':0,'Miss':1,'Mrs':3,'Other':4})

In [ ]:
train['Embarked'].unique()

In [ ]:
train['Embarked'].value_counts()

In [ ]:
train['Embarked']=train['Embarked'].map({'C':0,'Q':1,'S':2})
test['Embarked']=test['Embarked'].map({'C':0,'Q':1,'S':2})

In [ ]:
train['Embarked'].isnull().sum()

In [ ]:
train['Sex'].unique()

In [ ]:
train['Sex']=train['Sex'].map({'female':0,'male':1})
test['Sex']=test['Sex'].map({'female':0,'male':1})